# 6장. &#129303;Tokenizers 라이브러리
- [강좌링크](https://wikidocs.net/166814)

> 이 장에서 다룰 주제:
> - 새로운 텍스트 말뭉치를 기반으로 특정 체크포인트의 토크나이저와 유사한 새로운 토크나이저를 학습시키는 방법
> - fast tokenizer의 특징
> - NLP에서 사용되는 세 가지 주요 단어 토큰화 알고리즘의 차이점
> - &#129303;Tokenizers 라이브러리를 사용하여 처음부터 토크나이저를 구축하고 특정 데이터로 학습하는 방법


In [1]:
from custom_utils import *

from datasets import load_dataset
from transformers import AutoModelForTokenClassification, AutoModelForQuestionAnswering, AutoTokenizer, pipeline

import numpy as np

wrapper = CustomObject()

[2023-08-22 13:55:37,018] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


# 1. 기존 토크나이저에서 새로운 토크나이저 학습
각각의 배치에 대해 loss를 조금씩 더 작게 만드는 무작위 확률적 경사 하강법(stochastic gradient descent)으로 학습하는 모델과 달리 토크나이저 학습은 어떤 subword를 선택하는 것이 가장 좋은지 식별하려는 통계적 프로세스이며 결정론적(deterministic)이다.
> 동일한 말뭉치에서 동일한 알고리즘으로 학습 시 항상 동일한 결과를 얻을 수 있다.

## 말뭉치 모으기
&#129303;Transformers에는 기존에 존재하는 것들과 동일한 특성을 가진 새로운 토크나이저 학습 시 사용 가능한 매우 간단한 `AutoTokenizer.train_new_from_iterator()` API가 있다. 이를 실제로 실행하기 위해 GPT-2를 영어가 아닌 다른 언어로 학습한다고 가정한다. CodeSearchNet Challenge를 위해 생성된 CodeSearchNet 데이터셋을 로드한다.

In [3]:
wrapper.raw_datasets = load_dataset("code_search_net", "python")
wrapper.raw_datasets["train"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

위 데이터셋은 파이썬 코드에서 주석만 분리하고 있다. 토크나이저를 학습하기 위해 `whole_func_string` 칼럼만 사용한다.

In [6]:
wrapper.raw_datasets["train"][123456]["whole_func_string"]

'def get_new_token(self, netloc):\n        """Get a new token from BIG-IP and store it internally.\n\n        Throws relevant exception if it fails to get a new token.\n\n        This method will be called automatically if a request is attempted\n        but there is no authentication token, or the authentication token\n        is expired.  It is usually not necessary for users to call it, but\n        it can be called if it is known that the authentication token has\n        been invalidated by other means.\n        """\n        login_body = {\n            \'username\': self.username,\n            \'password\': self.password,\n        }\n\n        if self.auth_provider:\n            if self.auth_provider == \'local\':\n                login_body[\'loginProviderName\'] = \'local\'\n            elif self.auth_provider == \'tmos\':\n                login_body[\'loginProviderName\'] = \'tmos\'\n            elif self.auth_provider not in [\'none\', \'default\']:\n                providers 

가장 먼저 위 데이터셋을 텍스트 배치를 위해 텍스트 리스트로 만들고 한 번에 메모리에 로딩하지 않기 위해 iterator로 변환해 텍스트 리스트 리스트가 되어야 한다.

In [7]:
# wrapper.training_corpus = [
#     wrapper.raw_datasets["train"][i: i + 1000]["whole_func_string"] for i in range(0, len(wrapper.raw_datasets["train"]), 1000)
# ]

# 단 한 번만 사용할 수 있는 Python 제너레이터를 사용해 실제로 필요할 때까지 Python이 메모리에 아무것도 로드하지 않도록 설정
# 제너레이터 객체를 반환하는 함수로 작성
# def get_training_corpus():
#     return (
#         wrapper.raw_datasets["train"][i: i + 1000]["whols_func_string"]
#         for i in range(0, len(wrapper.raw_datasets["train"]), 1000)
#     )
# wrapper.training_corpus = get_training_corpus()

# 3가지 방법 중 마지막인 yeild 문 사용
def get_training_corpus():
    dataset = wrapper.raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx: start_idx + 1000]
        yield samples["whole_func_string"]

## 새로운 토크나이저 학습
이제 텍스트 배치의 이터레이터 형태의 말뭉치를 구성했으니 학습할 준비가 되었으므로 GPT-2 모델을 불러온다.

In [9]:
wrapper.old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

wrapper.example = """def add_numbers(a, b):
    '''Add the two numbers `a` and `b`.'''
    return a + b"""
wrapper.tokens = wrapper.old_tokenizer.tokenize(wrapper.example)
wrapper.tokens

['def',
 'Ġadd',
 '_',
 'n',
 'umbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 "Ġ'",
 "''",
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`',
 ".'",
 "''",
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

위 토크나이저는 공백과 줄바꿈을 나타내는 몇가지 특수 기호를 포함하고 있어 비효율적이다. 4칸 혹은 8칸의 공백을 개별 토큰으로 표현하고 _문자가 익숙하지 않은지 함수명이 이상하게 분할된다. 새로운 토크나이저를 학습하고 이러한 문제를 해결하는지 확인하기 위해 `train_new_from_iterator()`를 사용한다.

이는, fast tokenizer의 경우에만 동작한다.

대부분의 Transformer 모델에는 fast tokenizer가 있으며 **AutoTokenizer**의 경우 항상 fast tokenizer를 선택한다.

In [12]:
# text_iterator: generator of List[str]
# vocab_size: int

wrapper.tokenizer = wrapper.old_tokenizer.train_new_from_iterator(get_training_corpus(), 52000)

wrapper.tokens = wrapper.tokenizer.tokenize(wrapper.example)
print(wrapper.tokens)
print(len(wrapper.tokens))
print(len(wrapper.old_tokenizer.tokenize(wrapper.example)))

['def', 'Ġadd', '_', 'numbers', '(', 'a', ',', 'Ġb', '):', 'ĊĠĠĠ', "Ġ'''", 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', "`.'''", 'ĊĠĠĠ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']
27
37


In [14]:
wrapper.example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)
    
    def __call__(self, x):
        return x @ self.weights + self.bias
    """
print(wrapper.tokenizer.tokenize(wrapper.example))

del get_training_corpus
wrapper.init_wrapper()


GPU NVIDIA GeForce RTX 3060
memory occupied: 1.22GB
Allocated GPU Memory: 0.00GB
Reserved GPU Memory: 0.00GB


# 2. Fast Tokenizer의 특별한 능력

## Batch Encoding
토크나이저의 출력은 `BatchEncoding` 객체이다. 딕셔너리의 하위클래스지만 fast tokenizer에서 주로 사용하는 추가 메서드가 있다.

병렬화(parallelization) 외에도 범위(span)를 항상 추적하는데 이를 offset mapping이라 한다. 차례대로 각 단어를 생성된 토큰에 매핑하거나 원본 텍스트의 각 문자를 내부 토큰에 매핑하거나 그 반대로 매핑하는 것과 같은 기능들이다.

In [2]:
wrapper.tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
wrapper.example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
wrapper.encoding = wrapper.tokenizer(wrapper.example)
print("Tokenizer의 토큰화 결과 타입", type(wrapper.encoding))
print("tokenizer is fast") if wrapper.tokenizer.is_fast else print("tokenizer is slow")
print("encoding is fast") if wrapper.encoding.is_fast else print("encoding is slow")

# fast tokenizer로 가능한 것들

"""
1. 토큰 아이디를 다시 토큰으로 변환하지 않고도 토큰에 액세스할 수 있다.
"""
print("토큰 변환 없이 Access", wrapper.encoding.tokens())

"""
2. 각 토큰이 유래된 해당 단어의 인덱스를 가져올 수 있다.

이는 두 개의 토큰이 같은 단어에 있는지 아니면 토큰이 단어의 시작 부분에 있는지 확인하는데 특히 유용하다.

word_ids()와 마찬가지로 sentence_ids()를 통해 토큰을 가져온 문장에 해당 토큰을 매핑할 수도 있다.
"""
print("각 토큰이 유래된 해당 단어의 인덱스", wrapper.encoding.word_ids())

"""
3. 모든 단어 또는 토큰을 원본 텍스트의 문자에 매핑할 수 있다.
word_to_chars()
token_to_chars()
char_to_word()
char_to_token()
"""
wrapper.range = wrapper.encoding.word_to_chars(3)
print(wrapper.example[wrapper.range[0] : wrapper.range[1]])

Tokenizer의 토큰화 결과 타입 <class 'transformers.tokenization_utils_base.BatchEncoding'>
tokenizer is fast
encoding is fast
토큰 변환 없이 Access ['[CLS]', 'My', 'name', 'is', 'S', '##yl', '##va', '##in', 'and', 'I', 'work', 'at', 'Hu', '##gging', 'Face', 'in', 'Brooklyn', '.', '[SEP]']
각 토큰이 유래된 해당 단어의 인덱스 [None, 0, 1, 2, 3, 3, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, None]
Sylvain


## ***token-classification*** 파이프라인의 내부 동작
파이프라인의 세 단계 토큰화, 입력 전달, 후처리 중 `token-classification` 파이프라인의 후처리는 조금 더 복잡하다.
## 파이프라인으로 기본 실행 결과 도출하기
- NER 모델: dbmdz/bert-large-cased-finetuned-conll03-english

In [3]:
# Person, Organization, Location으로 식별
wrapper.token_classifier = pipeline("token-classification")
print(wrapper.token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn."))

"""
동일한 엔티티에 속하는 토큰을 그룹화
- simple: 개체명 내의 각 토큰에 대한 스코어의 평균
- first: 각 개체명의 스코어는 해당 개체명의 첫 번째 토큰의 스코어(Sylvain의 경우 토큰 S의 점수)
- max: 해당 엔티티 내의 토큰들 중 최대 스코어
- average: 해당 항목을 구성하는 단어(토큰 아님) 스코어의 평균
"""
wrapper.token_classifier = pipeline("token-classification", aggregation_strategy = "simple")
print(wrapper.token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn."))

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'entity': 'I-PER', 'score': 0.99938285, 'index': 4, 'word': 'S', 'start': 11, 'end': 12}, {'entity': 'I-PER', 'score': 0.99815494, 'index': 5, 'word': '##yl', 'start': 12, 'end': 14}, {'entity': 'I-PER', 'score': 0.99590707, 'index': 6, 'word': '##va', 'start': 14, 'end': 16}, {'entity': 'I-PER', 'score': 0.99923277, 'index': 7, 'word': '##in', 'start': 16, 'end': 18}, {'entity': 'I-ORG', 'score': 0.9738931, 'index': 12, 'word': 'Hu', 'start': 33, 'end': 35}, {'entity': 'I-ORG', 'score': 0.976115, 'index': 13, 'word': '##gging', 'start': 35, 'end': 40}, {'entity': 'I-ORG', 'score': 0.9887976, 'index': 14, 'word': 'Face', 'start': 41, 'end': 45}, {'entity': 'I-LOC', 'score': 0.9932106, 'index': 16, 'word': 'Brooklyn', 'start': 49, 'end': 57}]


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity_group': 'PER', 'score': 0.9981694, 'word': 'Sylvain', 'start': 11, 'end': 18}, {'entity_group': 'ORG', 'score': 0.9796019, 'word': 'Hugging Face', 'start': 33, 'end': 45}, {'entity_group': 'LOC', 'score': 0.9932106, 'word': 'Brooklyn', 'start': 49, 'end': 57}]


## inputs에서 predictions까지
위의 예제를 pipeline() 없이 동일한 결과 얻기

In [4]:
wrapper.model_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"
wrapper.tokenizer = AutoTokenizer.from_pretrained(wrapper.model_checkpoint)
wrapper.model = AutoModelForTokenClassification.from_pretrained(wrapper.model_checkpoint)

wrapper.example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
wrapper.inputs = wrapper.tokenizer(wrapper.example, return_tensors = "pt")
wrapper.outputs = wrapper.model(**wrapper.inputs)
print(wrapper.outputs)
print(wrapper.inputs["input_ids"].shape)
print(wrapper.outputs.logits.shape)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TokenClassifierOutput(loss=None, logits=tensor([[[ 8.7508, -2.2626, -1.5300, -2.2889, -0.6513, -2.0016, -0.0112,
          -2.0860,  0.3335],
         [ 8.4973, -2.3986, -1.3582, -2.7887,  0.7575, -1.8873,  0.4344,
          -1.9900, -0.3397],
         [ 9.4719, -2.2261, -0.9849, -2.6116,  0.1219, -2.0627, -0.1259,
          -1.8758, -0.0609],
         [ 9.8670, -2.2175, -1.3125, -2.4866, -0.2550, -1.8536,  0.0856,
          -1.7520, -0.6437],
         [-0.2011, -2.1873, -1.5316, -2.7110,  8.4025, -2.4168, -0.6980,
          -3.0337, -0.0997],
         [ 0.1065, -2.0520, -1.4787, -2.8139,  7.4525, -2.8399, -0.0626,
          -3.3666, -0.4683],
         [ 0.5985, -2.2538, -1.1926, -3.0111,  7.0070, -2.8675,  0.3492,
          -3.3129, -0.2878],
         [-0.0584, -2.2660, -1.4335, -3.1940,  8.3225, -2.6212, -0.0348,
          -2.9780, -0.2957],
         [ 9.6889, -2.4281, -1.5653, -2.5225, -0.9693, -1.5668,  0.4285,
          -1.9413, -0.6774],
         [ 9.0116, -2.1216, -1.4140, -2.69

모델에는 9개의 서로 다른 레이블이 존재하므로 1 X 19의 input이 1 X 19 X 9의 logit을 갖는다.

`softmax`를 사용해 logits를 확률로 변환하고 argmax를 사용해 예측 결과를 얻을 수 있다.(softmax는 순서를 변경하지 않기 때문에 logits에 대해서 argmax를 취할 수 있다.)

`model.config.id2label` 속성에는 예측 결과를 확인하는데 사용할 수 있는 레이블에 대한 인덱스 매핑이 포함되어있다.

In [6]:
wrapper.probabilities = torch.nn.functional.softmax(wrapper.outputs.logits, dim = -1)[0].tolist()
wrapper.predictions = wrapper.outputs.logits.argmax(dim=-1)[0].tolist()
print(wrapper.probabilities)
print(wrapper.predictions)

"""
- O: 개체명에 포함X, outside
- MISC: 기타, miscellaneous
- PER: 인물, person
- ORG: 조직, organization
- LOC: 지명, location

- B-XXX: 토큰이 엔티티의 시작부분에 있음을 나타냄
- I-XXX: 토큰이 엔티티의 내부에 있음을 나타냄
"""
print(wrapper.model.config.id2label)

[[0.9994322657585144, 1.6470316040795296e-05, 3.426706462050788e-05, 1.6042342394939624e-05, 8.250699465861544e-05, 2.1382335035013966e-05, 0.00015649119450245053, 1.965213414223399e-05, 0.0002208926307503134], [0.9989631175994873, 1.8515736883273348e-05, 5.240452446741983e-05, 1.2534720553958323e-05, 0.0004347364301793277, 3.087432560278103e-05, 0.00031468752422370017, 2.78607003565412e-05, 0.00014510865730699152], [0.999708354473114, 8.308118594868574e-06, 2.8745585950673558e-05, 5.6503527048334945e-06, 8.694847929291427e-05, 9.783467248780653e-06, 6.786138692405075e-05, 1.1793980775109958e-05, 7.241893035825342e-05], [0.9998350143432617, 5.645536475640256e-06, 1.3955152098787948e-05, 4.3133773033332545e-06, 4.017695027869195e-05, 8.123070074361749e-06, 5.648490696330555e-05, 8.99163478607079e-06, 2.7239138944423757e-05], [0.00018333422485738993, 2.515664164093323e-05, 4.8462032282259315e-05, 1.4900581845722627e-05, 0.9993828535079956, 1.999776031880174e-05, 0.00011153631203342229, 1

O로 분류되지 않은 각 토큰의 점수와 레이블만 가져오기

In [7]:
wrapper.results = []
wrapper.tokens = wrapper.inputs.tokens()

for idx, pred in enumerate(wrapper.predictions):
    label = wrapper.model.config.id2label[pred]
    if label != 'O':
        wrapper.results.append(
			{"entity": label, "score": wrapper.probabilities[idx][pred], "word": wrapper.tokens[idx]}
        )

wrapper.results

[{'entity': 'I-PER', 'score': 0.9993828535079956, 'word': 'S'},
 {'entity': 'I-PER', 'score': 0.9981548190116882, 'word': '##yl'},
 {'entity': 'I-PER', 'score': 0.995907187461853, 'word': '##va'},
 {'entity': 'I-PER', 'score': 0.9992327690124512, 'word': '##in'},
 {'entity': 'I-ORG', 'score': 0.9738931059837341, 'word': 'Hu'},
 {'entity': 'I-ORG', 'score': 0.9761149883270264, 'word': '##gging'},
 {'entity': 'I-ORG', 'score': 0.9887974858283997, 'word': 'Face'},
 {'entity': 'I-LOC', 'score': 0.99321049451828, 'word': 'Brooklyn'}]

위 결과는 파이프라인과 매우 유사하지만 한가지 차이점이 있다면 원본 문장에서 각 엔티티의 시작과 끝에 대한 정보를 제공한 파이프라인과 달리 위 코드는 그렇지않다.
Offset Mapping을 적용하기 위해 입력에 토크나이저 적용 시 `return_offsets_mapping=True`를 설정한다.

In [10]:
wrapper.inputs_with_offsets = wrapper.tokenizer(wrapper.example, return_offsets_mapping = True)

wrapper.results = []
wrapper.tokens = wrapper.inputs_with_offsets.tokens()
wrapper.offsets = wrapper.inputs_with_offsets["offset_mapping"]

for idx, pred in enumerate(wrapper.predictions):
    label = wrapper.model.config.id2label[pred]
    if label != 'O':
        start, end = wrapper.offsets[idx]
        wrapper.results.append(
            {
                "entity": label,
                "score": wrapper.probabilities[idx][pred],
                "word": wrapper.tokens[idx],
                "start": start,
                "end": end
            }
        )

wrapper.results

[{'entity': 'I-PER',
  'score': 0.9993828535079956,
  'word': 'S',
  'start': 11,
  'end': 12},
 {'entity': 'I-PER',
  'score': 0.9981548190116882,
  'word': '##yl',
  'start': 12,
  'end': 14},
 {'entity': 'I-PER',
  'score': 0.995907187461853,
  'word': '##va',
  'start': 14,
  'end': 16},
 {'entity': 'I-PER',
  'score': 0.9992327690124512,
  'word': '##in',
  'start': 16,
  'end': 18},
 {'entity': 'I-ORG',
  'score': 0.9738931059837341,
  'word': 'Hu',
  'start': 33,
  'end': 35},
 {'entity': 'I-ORG',
  'score': 0.9761149883270264,
  'word': '##gging',
  'start': 35,
  'end': 40},
 {'entity': 'I-ORG',
  'score': 0.9887974858283997,
  'word': 'Face',
  'start': 41,
  'end': 45},
 {'entity': 'I-LOC',
  'score': 0.99321049451828,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

## Entity 그룹화
엔티티 토큰들을 합쳐 단어를 만들때 ##를 지우고 ##로 시작하지 않으면 공백으로 연결하는 등의 지저분한 코드가 필요하지만 오프셋을 사용하는 경우 모든 사용자 정의 코드가 사라지고 slicing으로 해결 가능하다.
```python
example = "My name is Sylvain and I work at Hugging Face in Brooklyn"
example[33:45] # "Hugging Face"
```

특정 엔티티에 포함된 토큰들을 그룹화하는 동안 예측 결과를 후처리하는 코드를 작성하기 위해 B-XXX 또는 I-XXX로 레이블이 지정될 수 있는 첫 번째 엔티티를 제외하고 연속적이고 I-XXX로 레이블이 지정된 엔티티를 함께 그룹화한다. 따라서, 그룹화 도중 다음 토큰이 'O' 또는 B-XXX이거나 새로운 유형의 토큰으로 시작하는 경우 그룹화를 중지한다. 

In [12]:
wrapper.results = []
wrapper.inputs_with_offsets = wrapper.tokenizer(wrapper.example, return_offsets_mapping=True)
wrapper.tokens = wrapper.inputs_with_offsets.tokens()
wrapper.offsets = wrapper.inputs_with_offsets["offset_mapping"]

wrapper.idx = 0
while wrapper.idx < len(wrapper.predictions):
    pred = wrapper.predictions[wrapper.idx]
    label = wrapper.model.config.id2label[pred]
    if label == 'O':
        wrapper.idx += 1
        continue
    
    # Label의 "B-", "I-" 지우기
    label = label[2:]
    start, end = wrapper.offsets[wrapper.idx]
    
    # I-label 토큰 전부 모으기
    all_scores = []
    while (
        wrapper.idx < len(wrapper.predictions)
        and wrapper.model.config.id2label[wrapper.predictions[wrapper.idx]] == f"I-{label}"
    ):
        all_scores.append(wrapper.probabilities[wrapper.idx][pred])
        _, end = wrapper.offsets[wrapper.idx]
        wrapper.idx += 1
    
    # Score는 그룹 엔티티 내의 토큰 스코어의 평균
    score = np.mean(all_scores).item()
    word = wrapper.example[start:end]
    wrapper.results.append(
		{
            "entity_group": label,
            "score": score,
            "word": word,
            "start": start,
            "end": end
        }
    )
    wrapper.idx += 1

print(wrapper.results)

wrapper.init_wrapper()

[{'entity_group': 'PER', 'score': 0.998169407248497, 'word': 'Sylvain', 'start': 11, 'end': 18}, {'entity_group': 'ORG', 'score': 0.9796018600463867, 'word': 'Hugging Face', 'start': 33, 'end': 45}, {'entity_group': 'LOC', 'score': 0.99321049451828, 'word': 'Brooklyn', 'start': 49, 'end': 57}]

GPU NVIDIA GeForce RTX 3060
memory occupied: 0.97GB
Allocated GPU Memory: 0.00GB
Reserved GPU Memory: 0.00GB


# 3. QA 파이프라인에서의 fast tokenizer
`question-answering` 파이프라인을 살펴보고 오프셋을 활용하여 컨텍스트에서 입력 질문에 대한 답변을 직접 구하는 방법을 살펴본 후 절단(truncate)될 수밖에 없는 매우 긴 컨텍스트를 처리하는 방법을 본다.

## **question-answering** 파이프라인

In [2]:
wrapper.question_answerer = pipeline("question-answering")
wrapper.context = """
&#129303; Transformers is backed by the three most popular deep learning libraries - Jax, PyTorch, and TensorFlow - with a seamless integration between them. It`s straightforward to train your models with one before loading them for inference with the other.
"""

wrapper.question = "Which deep learning libraries back &#129303; Transformers?"
wrapper.question_answerer(question = wrapper.question, context = wrapper.context)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.9898537993431091,
 'start': 86,
 'end': 114,
 'answer': 'Jax, PyTorch, and TensorFlow'}

모델이 허용하는 최대 길이보다 긴 텍스트를 자르거나 분할할 수 없는 다른 파이프라인과 달리 이 파이프라인은 매우 긴 컨텍스트를 처리할 수 있으며 질문에 대한 답이 컨텍스트의 마지막에 있더라도 그 답변을 추출할 수 있다.

In [3]:
wrapper.long_context = """
 &#129303; Transformers: State of the Art NLP

 &#129303; Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction, question answering, summarization, translation, text generation and more in over 100 languages.
Its aim is to make cutting-edge NLP easier to use for everyone.

 &#129303; Transformers provides APIs to quickly download and use those pretrained models on a given text, fine-tune them on your own datasets and then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and can be modified to enable quick research experiments.
 
 Why should I use transformers?
 
 1. Easy-to-use state-of-the-art models:
    - High performance on NLU and NLG tasks.
    - Low barrier to entry for educators and practitioners.
    - Few user-facing abstractions with just three classes to learn.
    - A unified API for using all our pretrained models.
    - Lower compute costs, smaller carbon footprint:

2. Researchers can share trained models instead of always retraining.
    - Practitioners can reduce compute time and production costs.
    - Dozens of architectures with over 10,000 pretrained models, some in more than 100 languages.

3. Choose the right framework for every part of a model`s lifetime:
    - Train state-of-the-art models in 3 lines of code.
    - Move a single model between TF2.0/Pytorch frameworkd at will.
    - Seamlessly pick the right framework for training, evaluation and production.

4. Easily customize a model or an example to your needs:
    - We provide examples for each architecture to reproduce the results published by its original authors.
    - Model internals are exposed as consistently as possible.
    - Model files can be used independently of the library for quick experiments.

 &#129303; Transformers is backed by the three most popular deep learning libraries = Jax, PyTorch and TensorFlow - with a weamless integration between them. It`s straightforward to train your models with one before loading them for inference with the other.
"""
wrapper.question_answerer(question = wrapper.question, context = wrapper.long_context)

{'score': 0.9886583685874939,
 'start': 1958,
 'end': 1985,
 'answer': 'Jax, PyTorch and TensorFlow'}

## 질의 응답을 위한 사전학습 모델 사용하기
디폴트 체크포인트: distilbert-base-cased-distilled-squad

In [9]:
wrapper.model_checkpoint = "distilbert-base-cased-distilled-squad"
wrapper.tokenizer = AutoTokenizer.from_pretrained(wrapper.model_checkpoint)
wrapper.model = AutoModelForQuestionAnswering.from_pretrained(wrapper.model_checkpoint)

wrapper.inputs = wrapper.tokenizer(wrapper.question, wrapper.context, return_tensors = "pt")
wrapper.outputs = wrapper.model(**wrapper.inputs)

# 질의 응답 모델은 다른 모델과 다르게 하나의 로짓 텐서를 반환하지 않고 정답의 시작 토큰과 정답의 마지막 토큰에 해당하는 로짓 텐서를 반환한다.
wrapper.start_logits = wrapper.outputs.start_logits
wrapper.end_logits = wrapper.outputs.end_logits
print(wrapper.start_logits.shape, wrapper.end_logits.shape)

torch.Size([1, 77]) torch.Size([1, 77])


이러한 로짓들을 확률로 변환하기 위해 `softmax` 함수를 적용하기 전에 context가 아닌 토큰 인덱스를 masking해야 한다. 입력이 `[CLS] question [SEP] context [SEP]`이므로 질문에 포함된 토큰과 [SEP] 토큰을 마스킹하고 일부 모델에서는 context에 답이 없음을 나타내기 위해 사용할 수도 있으므로 [CLS] 토큰은 마스킹하지 않는다.

나중에 `softmax`를 적용할 것이기 때문에 masking하려는 로짓을 큰 음수로 바꾼다.

In [27]:
"""
[CLS] question [SEP] context [SEP]
 => [None, 0 ... 0, None, 1 ... 1, None]
- [CLS], [SEP] 토큰은 None
- 첫 번째 입력인 question은 0
- 두 번째 입력인 context는 1
"""
wrapper.sequence_ids = wrapper.inputs.sequence_ids()

# 컨텍스트 토큰들을 제외하고는 모두 마스킹한다.
wrapper.mask = [i != 1 for i in wrapper.sequence_ids]

# [CLS] 토큰은 마스킹하지 않는다.
wrapper.mask[0] = False
wrapper.mask = torch.tensor(wrapper.mask)[None]
# print(wrapper.mask)

# [CLS] 토큰과 context를 제외한 나머지 토큰들은 전부 -10000으로 masking
wrapper.start_logits[wrapper.mask] = -10000
wrapper.end_logits[wrapper.mask] = -10000

# masking을 완료했으니 `softmax`를 적용한다.
wrapper.start_probabilities = torch.nn.functional.softmax(wrapper.start_logits, dim=-1)[0]
wrapper.end_probabilities = torch.nn.functional.softmax(wrapper.end_logits, dim=-1)[0]
# print(f"softmax 적용 결과: {wrapper.start_probabilities}")

"""
start_probabilities로 예측한 시작 인덱스가 end_probabilities로 예측한 종료 인덱스보다 클 수 있으므로 추가 작업이 필요하다.

1. `start_idx <= end_idx`를 만족하는 가능한 start_idx 및 end_idx의 확률을 계산한 다음 가장 높은 확률을 가진 튜플(start_index, end_index)을 선택한다.
    - 답변이 start_index에서 시작해 end_index에서 끝날 확률은 두 확률의 곱과 같다.
    - 따라서, `start_index <= end_index`를 만족하는 모든 `start_index * end_index`를 계산하면 된다.
"""
# x = torch.tensor([1, 2, 3]) ---- tensor([1, 2, 3])
# y = x[:, None] ----------------- tensor([[1], [2], [3]])
# (77 X 77) = (77 X 1) X (1 X 77)
wrapper.scores = wrapper.start_probabilities[:, None] * wrapper.end_probabilities[None, :]
# print(f"start_index * end_index = {wrapper.scores}")

# start_index < end_index를 만족하는 값들을 0으로 설정하여 마스킹(나머지는 모두 양수).
# torch.triu() 함수는 인수로 전달된 2D tensor의 위쪽 삼각형 부분(삼각행렬)을 반환하므로 해당 마스킹을 수행할 수 있음.
wrapper.scores = torch.triu(wrapper.scores)
# print(f"masking: {wrapper.scores}")
print(f"삼각행렬 크기: {wrapper.scores.shape}")

삼각행렬 크기: torch.Size([77, 77])


In [29]:
# 최대값 인덱스 구하기
# argmax()를 이용해 최대값의 인덱스를 가져옴(PyTorch는 평탄화된 인덱스만 가져옴)
wrapper.max_index = wrapper.scores.argmax().item()

wrapper.start_index = wrapper.max_index // wrapper.scores.shape[1]
wrapper.end_index = wrapper.max_index % wrapper.scores.shape[1]
print(f"결과 scores: {wrapper.scores[wrapper.start_index, wrapper.end_index]}")

결과 scores: 0.98985356092453


응답들의 토큰 단위 `start_index`와 `end_index`를 구했기 때문에 이를 기반으로 이제 context 내 문자 단위 인덱스로 변환해야 한다. 여기서 offset이 매우 유용할 것이다.

In [31]:
wrapper.inputs_with_offsets = wrapper.tokenizer(wrapper.question, wrapper.context, return_offsets_mapping=True)
wrapper.offsets = wrapper.inputs_with_offsets["offset_mapping"]

wrapper.start_char, _ = wrapper.offsets[wrapper.start_index]
_, wrapper.end_char = wrapper.offsets[wrapper.end_index]

wrapper.result = {
    "answer": wrapper.context[wrapper.start_char:wrapper.end_char],
    "start": wrapper.start_char,
    "end": wrapper.end_char,
    "score": wrapper.scores[wrapper.start_index, wrapper.end_index]
}
print(wrapper.result)

{'answer': 'Jax, PyTorch, and TensorFlow', 'start': 86, 'end': 114, 'score': tensor(0.9899, grad_fn=<SelectBackward0>)}


## 길이가 긴 context 다루기
위에서 사용한 질문과 long_context를 토큰화해보면 `question-answering`파이프라인에서 사용된 최대 길이 384보다 더 많은 토큰들이 출력된다.

In [32]:
wrapper.inputs = wrapper.tokenizer(wrapper.question, wrapper.long_context)
print(len(wrapper.inputs["input_ids"]))

485


따라서 최대 길이만큼 입력을 truncate해야 하는데 이는 토크나이저에 `truncation="only_second"` 인수를 주면 된다. 하지만 이는 컨텍스트의 뒷부분에 정답이 있다면 답변을 찾을 수 없다. 이를 해결하기 위해 `question-answering`파이프라인은 context를 더 작은 청크로 분할하여 최대 길이를 지정할 수 있다. 정답을 찾을 수 있도록 context를 잘못된 위치에서 분할하지 않도록 하기 위해 청크 사이에 약간의 overlap도 포함한다. `return_overflowing_tokens=True`, `stride`인수로 겹침 정도를 지정할 수 있다.

`inputs["input_ids"]`의 각 항목이 최대 6개의 토큰을 갖는 청크로 분할하고 마지막 항목이 다른 항목과 같은 크기가 되도록 `padding`을 추가한다.

In [34]:
wrapper.sentence = "This sentence is not too long but we are going to split it anyway."
wrapper.inputs = wrapper.tokenizer(
    wrapper.sentence, truncation = True, return_overflowing_tokens=True, max_length=6, stride=2
)
for ids in wrapper.inputs["input_ids"]:
    print(wrapper.tokenizer.decode(ids))

print(wrapper.inputs.keys())

# 각 결과가 어느 문장에 해당하는지 알려주는 map
print(wrapper.inputs["overflow_to_sample_mapping"])

[CLS] This sentence is not [SEP]
[CLS] is not too long [SEP]
[CLS] too long but we [SEP]
[CLS] but we are going [SEP]
[CLS] are going to split [SEP]
[CLS] to split it anyway [SEP]
[CLS] it anyway. [SEP]
dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping'])
[0, 0, 0, 0, 0, 0, 0]


long_context로 돌아가보면 다음과 같다.

In [37]:
wrapper.inputs = wrapper.tokenizer(
    wrapper.question,
    wrapper.long_context,
    stride = 128,
    max_length = 384,
    padding = "longest",
    truncation = "only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True
)

# model에서 사용하지 않는 매개변수 제거
_ = wrapper.inputs.pop("overflow_to_sample_mapping")
wrapper.offsets = wrapper.inputs.pop("offset_mapping")

wrapper.inputs = wrapper.inputs.convert_to_tensors("pt")
print(wrapper.inputs["input_ids"].shape)

# 길이가 긴 컨텍스트는 두 개로 분할되었으며 모델의 출력은 시작 및 마지막 로짓으로 구성된다.
wrapper.outputs = wrapper.model(**wrapper.inputs)
wrapper.start_logits = wrapper.outputs.start_logits
wrapper.end_logits = wrapper.outputs.end_logits
print(wrapper.start_logits.shape, wrapper.end_logits.shape)

torch.Size([2, 384])
torch.Size([2, 384]) torch.Size([2, 384])


전과 마찬가지로 `softmax`를 취하기 전에 context의 일부가 아닌 토큰을 먼저 마스킹한다. 또한 모든 패딩 토큰을 마스킹한다.

In [59]:
wrapper.sequence_ids = wrapper.inputs.sequence_ids()

wrapper.mask = [i != 1 for i in wrapper.sequence_ids]

# Unmask [CLS]
wrapper.mask[0] = False

# Mask [PAD]
wrapper.mask = torch.logical_or(torch.tensor(wrapper.mask)[None], (wrapper.inputs["attention_mask"] == 0))

# -10000으로 masking
wrapper.start_logits[wrapper.mask] = -10000
wrapper.end_logits[wrapper.mask] = -10000

# softmax를 사용해 logits을 확률로 변환
wrapper.start_probabilities = torch.nn.functional.softmax(wrapper.start_logits, dim=-1)
wrapper.end_probabilities = torch.nn.functional.softmax(wrapper.end_logits, dim=-1)

다음 단계는 길이가 짧은 컨텍스트에 대해 수행한 작업과 유사하지만 청크가 2개이므로 2번 반복한다.

가능한 모든 답변에 점수를 부여한 다음 가장 좋은 점수를 받은 답변을 선택한다.

In [71]:
wrapper.candidates = []
for start_probs, end_probs in zip(wrapper.start_probabilities, wrapper.end_probabilities):
    scores = start_probs[:, None] * end_probs[None, :] # 384 X 384
    idx = torch.triu(scores).argmax().item()
    
    start_idx = idx // scores.shape[0]
    end_idx = idx % scores.shape[0]
    score = scores[start_idx, end_idx].item()
    
    wrapper.candidates.append((start_idx, end_idx, score))

print(wrapper.candidates)

[(23, 28, 0.7921538949012756), (202, 213, 0.9886583685874939)]


In [75]:
# 2개의 청크 보유
# candidates: (2, 3) offsets: (2, 384)
for candidate, offset in zip(wrapper.candidates, wrapper.offsets):
    start_token, end_token, score = candidate
    start_char, _ = offset[start_token]
    _, end_char = offset[end_token]
    result = {
        "answer": wrapper.long_context[start_char:end_char],
        "start": start_char,
        "end": end_char,
        "score": score
    }
    print(result)

wrapper.init_wrapper()

{'answer': 'State of the Art NLP', 'start': 26, 'end': 46, 'score': 0.7921538949012756}
{'answer': 'Jax, PyTorch and TensorFlow', 'start': 1958, 'end': 1985, 'score': 0.9886583685874939}

GPU NVIDIA GeForce RTX 3060
memory occupied: 1.01GB
Allocated GPU Memory: 0.00GB
Reserved GPU Memory: 0.00GB


# 4. Normalization 및 Pre-tokenization

## Normalization, 정규화
공백 제거, 고문자 변환 악센트 제거 등과 같은 일반적인 정제 작업이 포함된다.

In [77]:
# bert-base-cased 모델을 사용해 소문자 변환 후 악센트 제거
wrapper.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print(f"Backend Tokenizer: {type(wrapper.tokenizer.backend_tokenizer)}")
print(wrapper.tokenizer.backend_tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

Backend Tokenizer: <class 'tokenizers.Tokenizer'>
hello how are u?


## Pre-tokenization, 사전토큰화



In [80]:
wrapper.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print(f"bert-base-uncased: {wrapper.tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str('Hello, how are  you?')}")

wrapper.tokenizer = AutoTokenizer.from_pretrained("gpt2")
print(f"gpt2: {wrapper.tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str('Hello, how are  you?')}")

wrapper.tokenizer = AutoTokenizer.from_pretrained("t5-small")
print(f"t5-small: {wrapper.tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str('Hello, how are  you?')}")

wrapper.init_wrapper()

bert-base-uncased: [('Hello', (0, 5)), (',', (5, 6)), ('how', (7, 10)), ('are', (11, 14)), ('you', (16, 19)), ('?', (19, 20))]
gpt2: [('Hello', (0, 5)), (',', (5, 6)), ('Ġhow', (6, 10)), ('Ġare', (10, 14)), ('Ġ', (14, 15)), ('Ġyou', (15, 19)), ('?', (19, 20))]
t5-small: [('▁Hello,', (0, 6)), ('▁how', (7, 10)), ('▁are', (11, 14)), ('▁you?', (16, 20))]


## SentencePiece
BPE 토크나이저 섹션에서 볼 모든 모델과 함께 사용할 수 있는 텍스트 전처리를 위한 토큰화 알고리즘.

텍스트를 일련의 유니코드 문자로 간주하고 공백을 특수문자인 _로 치환. Unigram과 함께 사용하면 사전토큰화 단계가 필요하지 않으므로 공백 문자가 사용되지 않는 일본어나 중국어에 매우 유용하다.

SentencePiece의 또 다른 주요 기능은 가역적 토큰화이다. 공백에 대한 특별한 처리가 없기 때문에 토큰 디코딩은 토큰을 연결하고 _를 공백으로 바꾸는 것으로 간단히 수행되어 정규화된 텍스트가 출력된다.

## 토큰화 알고리즘 개요
![](../assets/tokenization.png)